# SCRAPING רשם החברות

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyodbc 
import sqlalchemy
import urllib

## create soup var

In [ ]:
r = requests.get("https://data.gov.il/api/action/datastore_search?resource_id=f004176c-b85f-4542-8901-7b3176f9a054&limit=0")
x = r.json()
j = x['result']
df = pd.DataFrame(j['records'])
sep = 10000
offset =0
while sep == 10000:
    r = requests.get("https://data.gov.il/api/action/datastore_search?resource_id=f004176c-b85f-4542-8901-7b3176f9a054&limit=10000&offset={}".format(offset))
    offset += 10000
    #print(r)
    x = r.json()
    #print(x.keys())
    j = x['result']
    df_batch = pd.DataFrame(j['records'])
    df = pd.concat([df,df_batch])
    sep = len(df_batch)

In [ ]:
nessesery_cols = ['מספר חברה',
'שם חברה',
'שם באנגלית',
'סוג תאגיד',
'סטטוס חברה',
'תאור חברה',
'מטרת החברה',
'תאריך התאגדות',
'חברה ממשלתית',
'מגבלות',
'מפרה',
'שנה אחרונה של דוח שנתי (שהוגש)',
'שם עיר',
'מיקוד',
'ת.ד.',
'מדינה',
'אצל']

#df[nessesery_cols]

export to sql

In [ ]:
params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=localhost;"
                                 "DATABASE=TempTests;"
                                 "UID=XXXXXXX;"
                                 "PWD=XXXXXXX")

engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
if len(df)>0:
    engine.execute(sqlalchemy.text('''TRUNCATE TABLE test''').execution_options(autocommit=True))

df.to_sql('test', con=engine, if_exists='append')

## selenium - doesnt work with this website

from selenium import webdriver
import time
path = "C:/Users/chen/Downloads/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)

options = webdriver.ChromeOptions(path)
options.add_argument('--ignore-certificate-errors')
options.add_argument("--test-type")
options.binary_location = "/usr/bin/chromium"
driver = webdriver.Chrome(path = path,chrome_options=options)
driver.get('https://data.gov.il/dataset/ica_companies/resource/f004176c-b85f-4542-8901-7b3176f9a054')

 click radio button
python_button = driver.find_elements_by_xpath("//*[@id='content']/div[3]/section/div[1]/div[1]/ul/li[2]/a")[0]
python_button.click()
